
### Importing libraries that will be useful :-

In [1]:
import warnings 
warnings.filterwarnings('ignore')
import json
import pandas as pd

# Question 6 solution : 

Reading data from DDW_PCA0000_2011_Indiastatedist.xlsx file i.e population data :-

In [2]:
population_data= pd.read_excel('DDW_PCA0000_2011_Indiastatedist.xlsx',usecols=['Level','Name','TRU','TOT_M','TOT_F'])
# population_data

Taking out district data from population data for calculating all the ratios at district level :-

In [4]:
census_data_district = population_data.loc[(population_data['Level'] == 'DISTRICT') & (population_data['TRU'] == 'Total')]
census_data_district = census_data_district.drop(columns=["TRU"])
census_data_district["Name"] = census_data_district["Name"].str.lower()
census_data_district = census_data_district.sort_values('Name',ignore_index=True)
# len(census_data_district) is 640

# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     print(census_data_district)
census_data_district

,Level,Name,TOT_M,TOT_F
0,DISTRICT,adilabad,1369597,1371642
1,DISTRICT,agra,2364953,2053844
2,DISTRICT,ahmedabad,3788051,3426174
3,DISTRICT,ahmednagar,2342825,2200334
4,DISTRICT,aizawl,199270,201039
...,...,...,...,...
635,DISTRICT,yadgir,590329,583942
636,DISTRICT,yamunanagar,646718,567487
637,DISTRICT,yanam,27301,28325
638,DISTRICT,yavatmal,1419965,1352383


Reading data from cowin_vaccine_data_districtwise.csv file i.e vaccine data for each district :- 

In [4]:
vaccine_data = pd.read_csv("cowin_vaccine_data_districtwise.csv",usecols=['State','District_Key','District',
                                                                '31/08/2021.5','31/08/2021.6'])
vaccine_data.rename(columns = {'31/08/2021.5':'Male(Individuals Vaccinated)',
                               '31/08/2021.6':'Female(Individuals Vaccinated)'}, inplace = True)
vaccine_data = vaccine_data.drop(vaccine_data.index[0])

## Making all the district names into small letters so that merging will be easy ##
vaccine_data["District"] = vaccine_data["District"].str.lower()

## Changing vaccination numbers values type from object to numeric ##
vaccine_data['Male(Individuals Vaccinated)'] = vaccine_data['Male(Individuals Vaccinated)'].astype(float)
vaccine_data['Female(Individuals Vaccinated)'] = vaccine_data['Female(Individuals Vaccinated)'].astype(float)
# vaccine_data['Female(Individuals Vaccinated)'] = pd.to_numeric(vaccine_data['Female(Individuals Vaccinated)'], errors='coerce')

## Sorting vaccination data on the basic of district values ##
vaccine_data = vaccine_data.sort_values('District',ignore_index=True)
# vaccine_data

So while printing "vaccine_data" we came to know that in vaccination data many districts are splitted into 2 or more parts. In order to avoid any confusion further while merging vaccination data with census data, lets merge them now (i.e. splitted districts)..

In [5]:
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     print(vaccine_data.groupby(['State','District_Key','District']).sum().reset_index().sort_values('District',ignore_index=True))
vaccine_data = vaccine_data.groupby(['State','District_Key','District']).agg({'Male(Individuals Vaccinated)':'sum','Female(Individuals Vaccinated)':'sum'}).reset_index()  
vaccine_data = vaccine_data.sort_values('District',ignore_index=True)
# len(vaccine_data) is 729
# vaccine_data

So now lets find out which dictricts are not common in both census data and vaccine data :- 

In [6]:
### From Census data fetching all the districts as a dataframe ###

district_in_census = census_data_district[['Name']].copy() 
district_in_census = district_in_census.sort_values('Name',ignore_index=True)
district_in_census.rename(columns = {'Name':'district_in_census'}, inplace = True)
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     print(district_in_census)

In [7]:
### From vaccine data fetching all the districts as a dataframe ###

district_in_vaccine = vaccine_data[['District']].copy() 
district_in_vaccine = district_in_vaccine.sort_values('District',ignore_index=True)
district_in_vaccine.rename(columns = {'District':'district_names_covid'}, inplace = True)
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     print(district_in_vaccine)

In [8]:
### From vaccination data pulling out all the districts as a list.. ###
district_in_vaccine_list = district_in_vaccine['district_names_covid'].tolist()
# district_in_vaccine_list                                                          

In [9]:
### Finding out all the districts from census data which are mismatching with vaccine data ###

district_in_census_not_in_covid_data = district_in_census[~district_in_census['district_in_census'].isin(district_in_vaccine_list)]
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     print(district_in_census_not_in_covid_data)
district_in_census_not_in_covid_data = district_in_census_not_in_covid_data.reset_index()
# district_in_census_not_in_covid_data

In [10]:
"""  So as we found all the mismatching districts, now lets make a dictionary with key as district's wrong spelling and
     value as its correct spelling.  """

corrected_districts = {}
corrected_districts['aizwal']='aizawl'
corrected_districts['anugul']='angul'
corrected_districts['ashok nagar']='ashoknagar'
corrected_districts['badgam']='budgam'
corrected_districts['baleshwar']='balasore'
corrected_districts['banas kantha']='banaskantha'
corrected_districts['bangalore rural']='bengaluru rural'
corrected_districts['bangalore urban']='bengaluru urban'
corrected_districts['baramula']='baramulla'
corrected_districts['baudh']='boudh'
corrected_districts['belgaum']='belagavi'
corrected_districts['bellary']='ballari'
corrected_districts['bemetara']='bametara'
corrected_districts['bid']='beed'
corrected_districts['bishwanath']='biswanath'
corrected_districts['chamarajanagar']='chamarajanagara'
corrected_districts['dantewada']='dakshin bastar dantewada'
corrected_districts['debagarh']='deogarh' 
corrected_districts['devbhumi dwaraka']='devbhumi dwarka'
corrected_districts['dhaulpur']='dholpur'
corrected_districts['east karbi anglong']='karbi anglong'
corrected_districts['faizabad']='ayodhya'
corrected_districts['fategarh sahib']='fatehgarh sahib'
corrected_districts['firozpur']='ferozepur'
corrected_districts['gondiya']='gondia'
corrected_districts['hugli']='hooghly'
corrected_districts['jagatsinghapur']='jagatsinghpur'
corrected_districts['jajapur']='jajpur'
corrected_districts['jalor']='jalore'
corrected_districts['jhunjhunun']='jhunjhunu'
corrected_districts['jyotiba phule nagar']='amroha'
corrected_districts['kabirdham']='kabeerdham'
corrected_districts['kaimur (bhabua)']='kaimur'
corrected_districts['kanchipuram']='kancheepuram'
corrected_districts['kheri']='lakhimpur kheri'
corrected_districts['kochbihar']='cooch behar'
corrected_districts['kodarma']='koderma'
corrected_districts['komram bheem']='komaram bheem'
corrected_districts['lahul and spiti']='lahaul and spiti'
corrected_districts['mahesana']='mehsana'
corrected_districts['mahrajganj']='maharajganj'
corrected_districts['maldah']='malda'
corrected_districts['marigaon']='morigaon'
corrected_districts['muktsar']='sri muktsar sahib'
corrected_districts['mumbai city']='mumbai'
corrected_districts['nandubar']='nandurbar'
corrected_districts['narsimhapur']='narsinghpur'
corrected_districts['nav sari']='navsari'
corrected_districts['pakaur']='pakur'
corrected_districts['palghat']='palakkad'
corrected_districts['panch mahal']='panchmahal'
corrected_districts['pashchim champaran']='west champaran'
corrected_districts['pashchimi singhbhum']='west singhbhum'
corrected_districts['pattanamtitta']='pathanamthitta'
corrected_districts['purba champaran']='east champaran'
corrected_districts['purbi singhbhum']='east singhbhum'
corrected_districts['puruliya']='purulia'
corrected_districts['rae bareilly']='rae bareli'
corrected_districts['rajauri']='rajouri'
corrected_districts['rangareddy']='ranga reddy'
corrected_districts['ri-bhoi']='ribhoi'
corrected_districts['sabar kantha']='sabarkantha'
corrected_districts['sahibzada ajit singh nagar']='s.a.s. nagar'
corrected_districts['sait kibir nagar']='sant kabir nagar'
corrected_districts['sant ravidas nagar']='bhadohi'
corrected_districts['sepahijala']='sipahijala'
corrected_districts['seraikela kharsawan']='saraikela-kharsawan'
corrected_districts['shaheed bhagat singh nagar']='shahid bhagat singh nagar'
corrected_districts['sharawasti']='shrawasti'
corrected_districts['shimoga']='shivamogga'
corrected_districts['shopian']='shopiyan'
corrected_districts['siddharth nagar']='siddharthnagar'
corrected_districts['sivagangai']='sivaganga'
corrected_districts['sonapur']='subarnapur'
corrected_districts['south salmara-mankachar']='south salmara mankachar'
corrected_districts['sri ganganagar']='ganganagar'
corrected_districts['sri potti sriramulu nellore']='s.p.s. nellore'
corrected_districts['the dangs']='dang'
corrected_districts['the nilgiris']='nilgiris'
corrected_districts['thoothukudi']='thoothukkudi'
corrected_districts['tiruchchirappalli']='tiruchirappalli'
corrected_districts['tirunelveli kattabo']='tirunelveli'
corrected_districts['tiruvanamalai']='tiruvannamalai'
corrected_districts['tumkur']='tumakuru'
corrected_districts['yadagiri']='yadgir'
corrected_districts['ysr']='y.s.r. kadapa'

# len(corrected_districts) # = 71
# corrected_districts

So now lets correct all the wrong spellings of districts in census data.

In [11]:
for i in census_data_district.index:
    a = ''
    a = census_data_district.iloc[i]['Name']
    census_data_district.at[i, 'Name'] = corrected_districts.get(a, a)
# with pd.option_context('display.max_rows', None,'display.max_columns', None):
#     display(census_data_district)
# census_data_district

Merging vaccination data and census data side by side to calculate ratios easily :- 

In [12]:
census_data_district['Name'] = census_data_district['Name'].str.strip()
vaccine_data['District'] = vaccine_data['District'].str.strip()
data_sample = pd.merge(vaccine_data,census_data_district,left_on='District', right_on='Name')
data_sample = data_sample.drop(columns=["Level","Name"])
# data_sample.rename(columns = {'District_x':'District_Name'}, inplace = True)
# data_sample = data_sample.sort_values('District_Name',ignore_index=True)
# with pd.option_context('display.max_rows', None,'display.max_columns', None):
#     display(data_sample)
# data_sample

Since in many states there are some distrits with same names so while merging they created some redundant rows which are incorrect, we found all of them by seeing manually. So now deleting those rows and re-setting index.
____Redundant rows are 30,31,81,217,218,450,451 and 466___

In [13]:
to_del = [30,31,81,217,218,450,451,466]
Final_data = data_sample.drop(data_sample.index[to_del])
Final_data = Final_data.reset_index()
Final_data = Final_data.drop(columns=["index"])
# with pd.option_context('display.max_rows', None,'display.max_columns', None):
#     display(Final_data)
# len(Final_data) is 630
# Final_data

Now as we have merged, cleaned and pre-processed data so now it is ready for calculating ratios :-

## Calculating ratios at district level :-

In [14]:
district_wise_data = Final_data.copy()
district_wise_data = district_wise_data.drop(columns=["District"])
district_wise_data.rename(columns = {'Male(Individuals Vaccinated)':'Males_Vaccinated',
                               'Female(Individuals Vaccinated)':'Females_Vaccinated',
                                 'District_Key':'districtid'}, inplace = True)
# district_wise_data

In [15]:
district_wise_data['vaccinationratio'] = district_wise_data['Females_Vaccinated']/district_wise_data['Males_Vaccinated']
district_wise_data['populationratio'] = district_wise_data['TOT_F']/district_wise_data['TOT_M']
district_wise_data['ratioofratios'] = district_wise_data['vaccinationratio']/district_wise_data['populationratio']
district_wise_data = district_wise_data.drop(columns=["State","Males_Vaccinated","Females_Vaccinated","TOT_M","TOT_F"])
district_wise_data = district_wise_data.sort_values('ratioofratios',ignore_index=True)
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     display(district_wise_data)
# district_wise_data

#### So finally saving ratios calculated at district level  as "district-vaccination-population-ratio.csv" :-

In [16]:
district_wise_data.to_csv('district-vaccination-population-ratio.csv', index=False)

## Calculating ratios at state level :-

In [17]:
State_wise_data = Final_data.copy()
State_wise_data = State_wise_data.drop(columns=["District","District_Key"])
State_wise_data.rename(columns = {'Male(Individuals Vaccinated)':'Males_Vaccinated',
                               'Female(Individuals Vaccinated)':'Females_Vaccinated'}, inplace = True)
# State_wise_data

In [18]:
State_wise_data = State_wise_data.groupby(['State']).agg({'Males_Vaccinated':'sum',
                                                          'Females_Vaccinated':'sum',
                                                          'TOT_M':'sum','TOT_F':'sum'}).reset_index()
# len(State_wise_data) is 36

In [6]:
State_wise_data['vaccinationratio'] = State_wise_data['Females_Vaccinated']/State_wise_data['Males_Vaccinated']
State_wise_data['populationratio'] = State_wise_data['TOT_F']/State_wise_data['TOT_M']
State_wise_data['ratioofratios'] = State_wise_data['vaccinationratio']/State_wise_data['populationratio']
State_wise_data = State_wise_data.drop(columns=["Males_Vaccinated","Females_Vaccinated","TOT_M","TOT_F"])
State_wise_data = State_wise_data.sort_values('ratioofratios',ignore_index=True)
# State_wise_data.head()
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     display(State_wise_data)
State_wise_data

NameError: name 'State_wise_data' is not defined

#### So finally saving ratios calculated at state level  as "state-vaccination-population-ratio.csv" :-

In [20]:
State_wise_data.to_csv('state-vaccination-population-ratio.csv', index=False)

## Calculating overall ratio :-

In [21]:
overall_data = pd.DataFrame()
overall_data['Country'] = ['India']
overall_data['vaccinationratio'] = Final_data['Female(Individuals Vaccinated)'].sum()/Final_data['Male(Individuals Vaccinated)'].sum()
overall_data['populationratio'] = Final_data['TOT_F'].sum()/Final_data['TOT_M'].sum()
overall_data['ratioofratios'] = overall_data['vaccinationratio']/overall_data['populationratio']
# overall_data
# with pd.option_context('display.max_rows', None,'display.max_columns', None,'display.precision', 4,):
#     display(overall_data)


#### So finally saving overall ratio calculated as "overall-vaccination-population-ratio.csv" :-

In [22]:
overall_data.to_csv('overall-vaccination-population-ratio.csv', index=False)

# Q6 completed..:)